In [2]:
import pandas as pd
import numpy as np
import csv
import math
import re
from tqdm import tqdm
from transformers import GPT2Tokenizer

## VALUE with Human Annotation

In [8]:
PRE_TRAINED_MODEL_NAME = 'EleutherAI/gpt-neo-1.3B'
tokenizer = GPT2Tokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [4]:
file_path = 'runs/08 Caleb Position/dataworks_sae_aave_validation_annotations_html.json'
output_path = 'runs/08 Caleb Position/data_1_0.5.csv'

In [6]:
df = pd.read_csv(output_path)
df.head()

,sae_prompt,sae_truth,aave_prompt,aave_truth
0,"There are many farms crowding the area,",which all produce milk and pork.,"There are many farms crowding the area,",which all produce milk and pork.
1,The toaster Tom had bought,from him didn't work.,The toaster Tom done bought,from him ain't work.
2,Where is there a very active tradition of,hunting of small to medium-sized wild game?,Where is it a very active tradition of,hunting of small to medium-sized wild game?
3,Bill is,very compassionate.,Bill is,very compassionate.
4,There are additional reporting,standards for financial audits.,It is additional reporting,standards for financial audits.


In [16]:
df['aave_truth'].iloc[189]

nan

In [5]:
df = pd.read_json(file_path)
df.head()

,"There are many farms crowding the area, which all produce milk and pork.",Her father kept bringing her various medical treatments and nothing had ever worked.,The president will run.,"tin foil hat for the sarg, what size do u wear?",The toaster Tom had bought from him didn't work.,Where is there a very active tradition of hunting of small to medium-sized wild game?,a rarity among recent iranian films : it 's a comedy full of gentle humor that chides the absurdity of its protagonist 's plight .,Bill is very compassionate.,There are additional reporting standards for financial audits.,"the film is not only a love song to the movies but it also is more fully an example of the kind of lush , all-enveloping movie experience it rhapsodizes",...,What is the best QB ranking that Cam Newton holds?,would have just gone more over-the-top instead of trying to have it both ways,"Ahern, who is travelling to Tokyo for an EU-Japan summit yesterday, will consult with other EU leaders about the election of the EU commission president.",Stocks finished lower Thursday.,there is not an ounce of honesty in the entire production .,The scientists appeared two years ago.,My son didn't become potty trained until about his 4th birthday.,"Inherited wealth may help explain why many Americans who have become rich may have had a ""substantial head start"".","Muriel has in her living room a famous painting of the ballerina Theresa Rubio by Cecile d'Arblay. It is very characteristic of her later work --- impetuous, almost crude, rather than realistic.",spain currently holds the osce's presidency.
sentence_html,<a href='dey/it' title='1'><mark>There</mark><...,Her father kept bringing her various medical t...,The president <a href='lexical' title='1'><mar...,"tin foil hat for the sarg, what size <a href='...",The toaster Tom <a href='been_done' title='1'>...,Where is <a href='dey/it' title='1'><mark>ther...,a rarity among recent iranian films : it 's a ...,Bill <a href='drop_aux' title='1'><mark>is</ma...,<a href='dey/it' title='1'><mark>There</mark><...,the film <a href='negative_concord' title='1'>...,...,What <a href='drop_aux' title='1'><mark>is</ma...,would <a href='been_done' title='1'><mark>have...,"Ahern, <a href='null_relcl' title='1'><mark>wh...",Stocks<a href='been_done' title='1'><mark></ma...,<a href='dey/it' title='1'><mark>there</mark><...,The scientists<a href='been_done' title='1'><m...,My son didn't become potty trained until about...,Inherited wealth may help explain why many Ame...,Muriel <a href='got' title='1'><mark>has</mark...,spain currently <a href='uninflect' title='1'>...
aave_synthetic,"It is many farms crowding the area, which all ...",Her father kept bringing her various medical t...,The president gon run.,"tin foil hat for the sarg, what size u wear?",The toaster Tom done bought from him ain't work.,Where is it a very active tradition of hunting...,a rarity among recent iranian films : it 's a ...,Bill very compassionate.,It is additional reporting standards for finan...,the film ain't only love no song to the movies...,...,What the best QB ranking Cam Newton hold,would done just gone more over-the-top instead...,"Ahern, is travelling to Tokyo for an EU-Japan ...",Stocks done finished lower Thursday.,dey ain't no ounce of honesty in the entire pr...,The scientists been appeared two years ago.,My son ain't become potty trained until about ...,Inherited wealth may help explain why many Ame...,Muriel got in her living room a famous paintin...,spain currently hold the osce presidency.
aave_human,"[There are many farms crowding the area, which...",[],[],[],[],[],[a rarity among recent iranian films : it 's a...,[Bill is very compassionate.],[],[The film is not only a love song to the movie...,...,[What's the best QB ranking Cam Newton hold],[woulda just gone more over-the-top instead of...,[],[Stocks finished lower Thursday],[],[The scientists had appeared two years ago.],[],[],[Muriel got a famous painting of the ballerina...,[]
s

In [5]:
df.shape

(17, 700)

In [118]:
sae = "In the unlikely event that the Electoral College throws the vote, how soon will we know?"
sae_split = sae.split()
print(' '.join(sae_split[:int(len(sae_split) * (2 / 3))]))

In the unlikely event that the Electoral College throws the


In [19]:
sae = "In the unlikely event that the Electoral College throws the vote, how soon will we know?"
html = "In the unlikely event <a href='null_relcl' title='1'><mark>that</mark></a>the Electoral College <a href='uninflect' title='2'><mark>throws</mark></a> the vote, how soon <a href='lexical' title='3'><mark>will</mark></a> we know?"
new_str = re.sub(r'<a href=.*?<mark>', '', html)
new_str = re.sub(r'</mark></a>', '', rep)
print(rep)



# prompt_ratio = (2 / 3)
# sae_split = sae.split()
# split_idx = len(' '.join(sae_split[:int(len(sae_split) * prompt_ratio)])) - 1
# num_occur = 2
# html_len = 0
# end_idx = 0
# for _ in range(num_occur):
#     match = re.search(r'<a href=.*?</a>', html[end_idx:])
#     print(match)
#     if match is None:
#         print('No!!')
#         break
#     else:
#         end_idx += match.end()
#         html_len += (match.end() - match.start() - len(re.findall(r'(?<=<mark>).*?(?=</mark>)', match.string)[0]))
#         print(len(re.findall(r'(?<=<mark>).*?(?=</mark>)', match.string)[0]))
# print(end_idx, html_len, split_idx)
# if end_idx - html_len <= split_idx:
#     print('YES!!!')
# else:
#     print('At last NO!!')


In the unlikely event thatthe Electoral College throws the vote, how soon will we know?


In [193]:
prompt_ratio = (2/3)
num_occur = 2
with open(output_path, 'a', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(['sae_prompt', 'sae_truth','aave_prompt', 'aave_truth'])
    for head in tqdm(df):
        accept_list = df[head]['accept']
        accept_list = [x for x in accept_list if x != None]
        if np.sum(accept_list) == len(accept_list) or df[head]['aave_human']:
            sae_split = head.split()
            split_idx = len(' '.join(sae_split[:math.ceil(len(sae_split) * prompt_ratio)]))
            html_len = 0
            end_idx = 0
            for _ in range(num_occur):
                match = re.search(r'<a href=.*?</a>', df[head]['sentence_html'][end_idx:])
                if match is None:
                    end_idx = 999999
                    break
                else:
                    end_idx += match.end()
                    html_len += (match.end() - match.start() - len(re.findall(r'(?<=<mark>).*?(?=</mark>)', match.string)[0]))
            if end_idx - html_len <= split_idx:
                if np.sum(accept_list) == len(accept_list):
                    aave_split = df[head]['aave_synthetic'].split()
                    aave_split_idx = len(' '.join(aave_split[:math.ceil(len(aave_split) * prompt_ratio)]))
                    writer.writerow([head[:split_idx], head[split_idx:], df[head]['aave_synthetic'][:aave_split_idx], df[head]['aave_synthetic'][aave_split_idx:]])
                else:
                    aave_split = df[head]['aave_human'][0].split()
                    aave_split_idx = len(' '.join(aave_split[:math.ceil(len(aave_split) * prompt_ratio)]))
                    writer.writerow([head[:split_idx], head[split_idx:], df[head]['aave_human'][0][:aave_split_idx], df[head]['aave_human'][0][aave_split_idx:]])

700it [00:00, 9586.73it/s]            


In [194]:
new_df = pd.read_csv(output_path)
print(new_df.shape)
counter = 0
for val in new_df['aave_prompt']:
    val = val.split()
    if len(val) >= 10:
        counter += 1
print(counter)

(208, 4)
137


## E-VALUE

In [7]:
df = pd.read_csv('runs/14/VALUE/RTE/train.tsv', encoding='utf-8', sep='\t')
df

,index,sentence1-glue,sentence1-glue-html,sentence1,sentence2-glue,sentence2-glue-html,sentence2,label,sentence1-ass,sentence1-been_done,...,sentence2-dey/it,sentence2-drop_aux,sentence2-got,sentence2-lexical,sentence2-negative_concord,sentence2-negative_inversion,sentence2-null_genetive,sentence2-null_relcl,sentence2-total,sentence2-uninflect
0,0,No Weapons of Mass Destruction Found in Iraq Yet.,No Weapons of Mass Destruction Found in Iraq Yet.,No Weapons of Mass Destruction Found in Iraq Yet.,Weapons of Mass Destruction Found in Iraq.,Weapons of Mass Destruction Found in Iraq.,Weapons of Mass Destruction Found in Iraq.,not_entailment,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
1,1,"A place of sorrow, after Pope John Paul II die...","A place of sorrow, after Pope John Paul II die...","A place of sorrow, after Pope John Paul II die...",Pope Benedict XVI is the new leader of the Rom...,Pope Benedict XVI <a href='drop_aux' title='1'...,Pope Benedict XVI the new leader of da Roman C...,entailment,0,0,...,0,1,0,1,0,0,0,0,2.0,0.0
2,2,Herceptin was already approved to treat the si...,Herceptin was already approved to treat the si...,Herceptin was already approved to treat the si...,Herceptin can be used to treat breast cancer.,Herceptin can be used to treat breast cancer.,Herceptin can be used to treat breast cancer.,entailment,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
3,3,"Judie Vivian, chief executive at ProMedica, a ...","Judie Vivian, chief executive at ProMedica, a ...","Judie Vivian, chief executive at ProMedica, a ...",The previous name of Ho Chi Minh City was Saigon.,The previous name of Ho Chi Minh City was Saigon.,Da previous name of Ho Chi Minh City was Saigon.,entailment,0,1,...,0,0,0,1,0,0,0,0,1.0,0.0
4,4,A man is due in court later charged with the m...,A man <a href='drop_aux' title='1'><mark>is</m...,A man due in court later charged w/ the murder...,Paul Stewart Hutchinson is accused of having s...,Paul Stewart Hutchinson is accused of <a href=...,Paul Stewart Hutchinson is accused of stabbed ...,not_entailment,0,1,...,0,1,0,1,0,0,0,0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,2485,There is none. They found as many weapons in t...,<a href='dey/it' title='1'><mark>There</mark><...,Dey ain't none. They found as many weapons in ...,Weapons of mass destruction found in Iraq.,Weapons of mass destruction found in Iraq.,Weapons of mass destruction found in Iraq.,not_entailment,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
2486,2486,"Dr. Eric Goosby, a pioneer in the fight agains...","Dr. Eric Goosby, a pioneer in the fight agains...","Dr. Eric Goosby, a pioneer in the fight agains...",Pepfar is committed to fighting AIDS.,Pepfar is committed to fighting AIDS.,Pepfar is committed to fightin AIDS.,entailment,0,1,...,0,0,0,1,0,0,0,0,1.0,0.0
2487,2487,"NASA's Saturn exploration spacecraft, Cassini ...",NASA<a href='null_genetive' title='1'><mark>'s...,"NASA Saturn exploration spacecraft, Cassini , ...",Titan is the fifteenth of Saturn's known satel...,Titan <a href='drop_aux' title='1'><mark>is</m...,Titan the fifteenth of Saturn known satellites.,not_entailment,0,1,...,0,1,0,0,0,0,1,0,2.0,0.0
2488,2488,Brooklyn Borough Hall featured a Who's Who in ...,Brooklyn Borough Hall featured a Who's Who in ...,Brooklyn Borough Hall featured a Who's Who in ...,The Brooklyn Book Festival is held in Brooklyn...,The Brooklyn Book Festival is held in Brooklyn...,The Brooklyn Book Festival is held in Brooklyn...,entailment,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0


In [8]:
PRE_TRAINED_MODEL_NAME = 'EleutherAI/gpt-neo-1.3B'
tokenizer = GPT2Tokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
def count_aave_features(s):
    return len(re.findall(r'<a href=.*?<mark>', s))

In [28]:
total_len = 0
count = 0

new_df = pd.DataFrame()
df = pd.read_csv('runs/14/dataset/RTE_2fts.csv', encoding='utf-8')
for i in range(len(df)):
    total_len += len(tokenizer.tokenize(df['aave_prompt'].iloc[i]))
    count += 1

total_len / count

55.76063829787234

In [29]:
output_path = 'runs/14/dataset/RTE_sentence2_2fts.csv'
df = pd.read_csv('runs/14/VALUE/RTE/train.tsv', encoding='utf-8', sep='\t')
prompt_ratio = (2/3)
num_occur = 2
with open(output_path, 'a', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(['sae_prompt', 'sae_truth','aave_prompt', 'aave_truth', 'sae_html'])
    for i in range(len(df)):
        sae_split = df['sentence2-glue'].iloc[i].split()
        split_idx = len(' '.join(sae_split[:math.ceil(len(sae_split) * prompt_ratio)]))
        html_len = 0
        end_idx = 0
        for _ in range(num_occur):
            match = re.search(r'<a href=.*?</a>', df['sentence2-glue-html'].iloc[i][end_idx:])
            if match is None:
                end_idx = 999999
                break
            else:
                end_idx += match.end()
                html_len += (match.end() - match.start() - len(re.findall(r'(?<=<mark>).*?(?=</mark>)', match.string)[0]))
        if end_idx - html_len <= split_idx:
            aave_split = df['sentence2'].iloc[i].split()
            aave_split_idx = len(' '.join(aave_split[:math.ceil(len(aave_split) * prompt_ratio)]))
            writer.writerow([df['sentence2-glue'].iloc[i][:split_idx], df['sentence2-glue'].iloc[i][split_idx:], df['sentence2'].iloc[i][:aave_split_idx], df['sentence2'].iloc[i][aave_split_idx:], df['sentence2-glue-html'].iloc[i]])

In [30]:
df = pd.read_csv('runs/14/dataset/RTE_sentence2_2fts.csv')
df

,sae_prompt,sae_truth,aave_prompt,aave_truth,sae_html
0,Microsoft's Hotmail has raised its storage,capacity to 250MB.,Microsoft Hotmail done raised its storage,capacity to 250MB.,Microsoft<a href='null_genetive' title='1'><ma...
1,North Korea says it has a stockpile of nuclear...,and is building more.,North Korea say it got a stockpile of nuclear ...,and is building more.,North Korea <a href='uninflect' title='1'><mar...
2,It seems unlikely that there will be a coaliti...,and Angela Merkel's Christian Democratic Union.,It seem unlikely that it gon be a coalition be...,and Angela Merkel Christian Democratic Union.,It <a href='uninflect' title='1'><mark>seems</...
3,The soldiers were driving a civilian car and w...,took place between them and Iraqi patrols.,Da soldiers was driving a civilian whip and wa...,took place between them and Iraqi patrols.,The soldiers <a href='uninflect' title='1'><ma...
4,Bomb-sniffing dogs were estimated at Rodriguez's,Mulberry St. apartment.,Bomb-sniffing dogs was estimated at Rodriguez,Mulberry St. apartment.,Bomb-sniffing dogs <a href='uninflect' title='...
...,...,...,...,...,...
104,Houston is George Zamka's,hometown.,Houston George Zamka,hometown.,Houston <a href='drop_aux' title='1'><mark>is<...
105,Smoking marijuana does not increase the risk of,developing oral cancer.,Smoking marijuana do not increase no risk of,developing oral cancer.,Smoking marijuana <a href='uninflect' title='1...
106,"Kozlowski and Swartz, who were found guilty of...","on Monday, to up to 25 years in prison.","Kozlowski and Swartz, was found guilty of stea...","on Monday, to up to 25 years in prison.","Kozlowski and Swartz, <a href='null_relcl' tit..."
107,Pluto's trip around the Sun lasts,250 years.,Pluto trip around the Sun last,250 years.,Pluto<a href='null_genetive' title='1'><mark>'...
